In [1]:
import pyarrow # must occur prior to ray import
import ray
from ray import tune
from ray.tune import ExperimentAnalysis
from ray.tune.search.hyperopt import HyperOptSearch
import datetime
import numpy as np
import pandas as pd
import random
import seaborn as sns; sns.set()
from collections import Counter
from datasets import load_from_disk
from scipy.stats import ranksums
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments

from geneformer import DataCollatorForCellClassification
from geneformer import TranscriptomeTokenizer
import anndata as ad
import pandas as pd

2023-11-17 16:55:38,650	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-17 16:55:39,083	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.7.0 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
train_dataset=load_from_disk("/root/autodl-tmp/FB.dataset")

In [3]:
num_proc=30
train_dataset

Dataset({
    features: ['input_ids', 'cell_type', 'condition', 'length'],
    num_rows: 2182
})

In [4]:
conditions = train_dataset["condition"]
target_names = list(set(conditions))

In [5]:
target_names

['Ctrl', 'AF']

In [6]:
target_name_id_dict = dict(zip(target_names,[i for i in range(len(target_names))]))

In [7]:
target_name_id_dict

{'Ctrl': 0, 'AF': 1}

In [8]:
trainset_v3 = train_dataset.rename_column("condition","label")

In [9]:
trainset_v3

Dataset({
    features: ['input_ids', 'cell_type', 'label', 'length'],
    num_rows: 2182
})

In [10]:
def classes_to_ids(example):
    example["label"] = target_name_id_dict[example["label"]]
    return example

trainset_v4 = trainset_v3.map(classes_to_ids, num_proc=num_proc)

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [11]:
indiv_list = trainset_v4["input_ids"]

In [12]:
len(indiv_list)

2182

In [13]:
random.seed(42)
train_indiv = random.sample(indiv_list,round(0.7*len(indiv_list)))
eval_indiv = [indiv for indiv in indiv_list if indiv not in train_indiv]
valid_indiv = random.sample(eval_indiv,round(0.5*len(eval_indiv)))
test_indiv = [indiv for indiv in eval_indiv if indiv not in valid_indiv]

In [14]:
def if_train(example):
    return example["input_ids"] in train_indiv

classifier_trainset = trainset_v4.filter(if_train,num_proc=num_proc).shuffle(seed=42)

In [15]:
def if_valid(example):
    return example["input_ids"] in valid_indiv

classifier_validset = trainset_v4.filter(if_valid,num_proc=num_proc).shuffle(seed=42)

In [16]:
def if_test(example):
    return example["input_ids"] in test_indiv

classifier_testdset = trainset_v4.filter(if_test,num_proc=num_proc).shuffle(seed=42)

In [17]:
current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}{current_date.month:02d}{current_date.day:02d}"
output_dir = f"/root/autodl-tmp/FB/model/{datestamp}_geneformer_DiseaseClassifier/"

In [18]:
import os
saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
if os.path.isfile(saved_model_test) == True:
    raise Exception("Model already saved to this directory.")

In [19]:
# make output directory
import subprocess
subprocess.call(f'mkdir {output_dir}', shell=True)

mkdir: cannot create directory ‘/root/autodl-tmp/FB/model/231116_geneformer_DiseaseClassifier/’: File exists


1

In [20]:
# set training parameters
# how many pretrained layers to freeze
freeze_layers = 2
# batch size for training and eval
geneformer_batch_size = 12
# number of epochs
epochs = 10
# logging steps
logging_steps = round(len(classifier_trainset)/geneformer_batch_size/10)

In [21]:
def model_init():
    model = BertForSequenceClassification.from_pretrained("/root/autodl-tmp/Geneformer/",
                                                          num_labels=len(target_names),
                                                          output_attentions = False,
                                                          output_hidden_states = False)
    if freeze_layers is not None:
        modules_to_freeze = model.bert.encoder.layer[:freeze_layers]
        for module in modules_to_freeze:
            for param in module.parameters():
                param.requires_grad = False

    return model

In [22]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    # calculate accuracy and macro f1 using sklearn's function
    acc = accuracy_score(labels, preds)
    macro_f1 = f1_score(labels, preds, average='macro')
    return {
      'accuracy': acc,
      'macro_f1': macro_f1
    }

In [23]:
training_args = {
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "steps",
    "eval_steps": logging_steps,
    "logging_steps": logging_steps,
    "group_by_length": True,
    "length_column_name": "length",
    "disable_tqdm": True,
    "skip_memory_metrics": True, # memory tracker causes errors in raytune
    "per_device_train_batch_size": geneformer_batch_size,
    "per_device_eval_batch_size": geneformer_batch_size,
    "num_train_epochs": epochs,
    "load_best_model_at_end": True,
    "output_dir": output_dir,
    "save_steps":13,
}

In [24]:
training_args_init = TrainingArguments(**training_args)

In [25]:
trainer = Trainer(
    model_init=model_init,
    args=training_args_init,
    data_collator=DataCollatorForCellClassification(),
    train_dataset=classifier_trainset,
    eval_dataset=classifier_validset,
    compute_metrics=compute_metrics,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
ray_config = {
    "num_train_epochs": tune.choice([epochs]),
    "learning_rate": tune.loguniform(1e-6, 1e-3),
    "weight_decay": tune.uniform(0.0, 0.3),
    "lr_scheduler_type": tune.choice(["linear","cosine","polynomial"]),
    "warmup_steps": tune.uniform(100, 2000),
    "seed": tune.uniform(0,100),
    "per_device_train_batch_size": tune.choice([geneformer_batch_size])
}

In [27]:
hyperopt_search = HyperOptSearch(
    metric="eval_accuracy", mode="max")

In [28]:
# optimize hyperparameters
trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    resources_per_trial={"cpu":8,"gpu":1},
    hp_space=lambda _: ray_config,
    search_alg=hyperopt_search,
    n_trials=100, # number of trials
    progress_reporter=tune.CLIReporter(max_report_frequency=600,
                                                   sort_by_metric=True,
                                                   max_progress_rows=100,
                                                   mode="max",
                                                   metric="eval_accuracy",
                                                   metric_columns=["loss", "eval_loss", "eval_accuracy","macro_f1"])
)

DeprecationWarning: Accepting a `checkpoint_dir` argument in your training function is deprecated.
Please use `ray.train.get_checkpoint()` to access your checkpoint as a
`ray.train.Checkpoint` object instead. See below for an example:

Before
------

from ray import tune

def train_fn(config, checkpoint_dir=None):
    if checkpoint_dir:
        torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

After
-----

from ray import train, tune

def train_fn(config):
    checkpoint: train.Checkpoint = train.get_checkpoint()
    if checkpoint:
        with checkpoint.as_directory() as checkpoint_dir:
            torch.load(os.path.join(checkpoint_dir, "checkpoint.pt"))
    ...

tuner = tune.Tuner(train_fn)
tuner.fit()

In [30]:
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/Geneformer/ and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'classifier.bias', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'loss': 0.6876, 'learning_rate': 4.94921875e-05, 'epoch': 0.1}
{'eval_loss': 0.6455250978469849, 'eval_accuracy': 0.6615853658536586, 'eval_macro_f1': 0.5037009773851879, 'eval_runtime': 2.2171, 'eval_samples_per_second': 147.939, 'eval_steps_per_second': 12.629, 'epoch': 0.1}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.6559, 'learning_rate': 4.8984375e-05, 'epoch': 0.2}
{'eval_loss': 0.6428956985473633, 'eval_accuracy': 0.6646341463414634, 'eval_macro_f1': 0.5332953898690951, 'eval_runtime': 2.2209, 'eval_samples_per_second': 147.687, 'eval_steps_per_second': 12.607, 'epoch': 0.2}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


...

/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'loss': 0.2176, 'learning_rate': 2.34375e-07, 'epoch': 9.95}
{'eval_loss': 0.5530338883399963, 'eval_accuracy': 0.7713414634146342, 'eval_macro_f1': 0.7522733452161565, 'eval_runtime': 2.2315, 'eval_samples_per_second': 146.988, 'eval_steps_per_second': 12.548, 'epoch': 9.95}


/root/miniconda3/lib/python3.8/site-packages/geneformer/collator_for_classification.py:581: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


{'train_runtime': 448.8071, 'train_samples_per_second': 34.024, 'train_steps_per_second': 2.852, 'train_loss': 0.38353993808850645, 'epoch': 10.0}


TrainOutput(global_step=1280, training_loss=0.38353993808850645, metrics={'train_runtime': 448.8071, 'train_samples_per_second': 34.024, 'train_steps_per_second': 2.852, 'train_loss': 0.38353993808850645, 'epoch': 10.0})

In [31]:
predictions = trainer.predict(classifier_testdset)

In [32]:
import pickle
with open(f"{output_dir}predictions.pickle", "wb") as fp:
        pickle.dump(predictions, fp)
trainer.save_metrics("eval",predictions.metrics)
trainer.save_model(output_dir)

In [2]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats

In [3]:
isp = InSilicoPerturber(perturb_type="delete",
                        perturb_rank_shift=None,
                        genes_to_perturb="all",
                        combos=0,
                        anchor_gene=None,
                        model_type="CellClassifier",
                        num_classes=2,
                        emb_mode="cell",
                        cell_emb_style="mean_pool",
                        filter_data=None,  #{"cell_type":["CD8-positive, alpha-beta T cell"]}
                        cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []},
                        max_ncells=1000,
                        emb_layer=0,
                        forward_batch_size=50,
                        nproc=4)

isp.perturb_data("/root/autodl-tmp/FB/model/231116_geneformer_DiseaseClassifier",
                 "/root/autodl-tmp/FB.dataset",
                 "/root/autodl-tmp/FB/",
                 "FB_perturbation")

Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

/root/miniconda3/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


Filter (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter (num_proc=4):   0%|          | 0/2182 [00:00<?, ? examples/s]

  0%|          | 0/852 [00:00<?, ?it/s]

...

Map (num_proc=4):   0%|          | 0/368 [00:00<?, ? examples/s]

Map:   0%|          | 0/334 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/334 [00:00<?, ? examples/s]

Map:   0%|          | 0/294 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/294 [00:00<?, ? examples/s]

In [4]:
ispstats = InSilicoPerturberStats(mode="goal_state_shift",
                                  genes_perturbed="all",
                                  combos=0,
                                  anchor_gene=None,
                                  cell_states_to_model={'state_key': 'condition',
                                              'start_state': 'AF', 
                                              'goal_state': 'Ctrl', 
                                              'alt_states': []})

In [5]:
ispstats.get_stats("/root/autodl-tmp/FB",
                   None,
                   "/root/autodl-tmp/FB/",
                   "FB")

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/13500 [00:00<?, ?it/s]

  0%|          | 0/13500 [00:00<?, ?it/s]